In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import datetime
from openpyxl import load_workbook
from openpyxl.chart import LineChart, Reference
from openpyxl.chart.label import DataLabelList
from openpyxl.styles import Border, Font, Side, Alignment


#iestatījumi
nosauk=r'C:\Users\Arts\Desktop\uzdprogr\Laikapstakliscam.xlsx'  #izveidota excel faila nosaukums(un atrasanas vieta)
laiks=0 #laiks starp cikliem(minūtēs)
ciklusk=1 #cik ciklus veiks skripts


parveidlaiks=laiks*60

skaits=0
mazskaits=0

cikls=0
iepdatulaiks=[]
iepdminl=[]
iepdmaxl=[]
iepmintemp=[]
iepmaxtemp=[]

tempdatulaiks=[]
tempdminl=[]
tempdmaxl=[]
tempmintemp=[]
tempmaxtemp=[]
laiki=[]
cikli=[]

sec=':00'

def tabula():
    global cikls
    global df
    global iepdatulaiks
    global iepdminl
    global iepdmaxl
    global iepmintemp
    global iepmaxtemp
    global mazskaits
    global tempdatulaiks
    global tempdminl
    global tempdmaxl
    global tempmintemp
    global tempmaxtemp
    global skaits
    global laiki
    global sec
    global cikli
    
    html=requests.get('https://www.meteolapa.lv/laika-apstakli').text
    soup=BS(html, 'lxml')

    #rindinas
    rindas=soup.select('tr.station-row')

    #rindu sk

    if cikls==0:
        for j in rindas:
            if 'LV Ceļi' in rindas[skaits].text:
                mazskaits=mazskaits+1

            skaits=skaits+1
            
        j=0
        while j!=skaits:
            iepdatulaiks.append(' ')
            iepdminl.append(' ')
            iepdmaxl.append(' ')
            iepmintemp.append(' ')
            iepmaxtemp.append(' ')
            j=j+1

    #tabulas dati (10 kollonas)
    tdati={'Vieta':[],
          'Avots':[],
          'Temperatūra(°)':[],
          'Laikapstākļi':[],
          'Vēja ātrums':[],
          'Diennakts minimālā temperatūra(°)':[],
          'Diennakts maksimālā temperatūra(°)':[],
          'Minimālā temperatura(°)':[],
          'Maksimālā temperatura(°)':[],
          'Datu atjaunošanas laiks':[]}

    df=pd.DataFrame(tdati)
    dati=[]
    
    #filtracijas sistema
    i=0
    x=0
    mazcikls=0
    for j in rindas:
        if 'LV Ceļi' in rindas[i].text:
            linija=rindas[i].text.splitlines()
            dati.clear()
            #atseviski elementi
            
            #Vieta
            vieta=linija[3]
            dati.append(vieta)#1
            
            #Avots
            avots=linija[5]
            dati.append(avots)#2
            
            #Laiks1
            datulaiks=linija[6]
            datulaiks=datulaiks+sec
            laiki.append(datulaiks)
            if datulaiks==iepdatulaiks[x]:
                l=0
                while l!=3:
                    dati.append('')
                    l=l+1
                    
            else:

                #Temperatura
                temp=linija[7]
                if 'nav' in temp:
                    temp=''
                else:
                    temp=temp.replace('°','')
                    temp=float(temp)
                dati.append(temp)#3

                #Laikapstakli
                laikapst=linija[9]
                if 'nav datu' not in laikapst:
                    dati.append(laikapst)#4
                else:
                    dati.append('')

                #Veja atrums
                vejaatr=linija[10]
                if 'nav datu' not in vejaatr:
                    dati.append(vejaatr)#5
                else:
                    dati.append('')#5
                
            #Diennaksts minimala temp
            dmin=linija[11].split()
            dmint=dmin[0]
            dminl=dmin[1]
            if 'nav' in dmint:
                dmint=''
                
            else:
                dmint=dmint.replace('°','')
                dmint=float(dmint)
                
            if dminl==iepdminl[x]:
                dati.append('')
                
            else:
                dati.append(dmint)#6
            
            #Diennaksts maximala temp
            dmax=linija[12].split()
            dmaxt=dmax[0]
            dmaxl=dmax[1]
            if 'nav' in dmaxt:
                dmaxt=''
                
            else:
                dmaxt=dmaxt.replace('°','')
                dmaxt=float(dmaxt)
                
            if dmaxl==iepdmaxl[x]:
                dati.append('')
                
            else:
                dati.append(dmaxt)#7
            
            #Minimala temp
            mintemp=linija[13]
            mintemp=mintemp.replace('°','')
            mintemp=float(mintemp)
            if mintemp==iepmintemp[x]:
                dati.append('')
                
            else:
                dati.append(mintemp)#8

            #Maximala temp
            maxtemp=linija[14]
            maxtemp=maxtemp.replace('°','')
            maxtemp=float(maxtemp)
            if maxtemp==iepmaxtemp[x]:
                dati.append('')
                
            else:
                dati.append(maxtemp)#9

            #Laiks2
            dati.append(datulaiks)#10
            
            tempdatulaiks.append(datulaiks)
            tempdminl.append(dminl)
            tempdmaxl.append(dmaxl)
            tempmintemp.append(mintemp)
            tempmaxtemp.append(maxtemp)
            
            mazcikls=mazcikls+1
            if mazskaits==mazcikls:
                iepdatulaiks.clear()
                iepdminl.clear()
                iepdmaxl.clear()
                iepmintemp.clear()
                iepmaxtemp.clear()
                
                iepdatulaiks.extend(tempdatulaiks)
                iepdminl.extend(tempdminl)
                iepdmaxl.extend(tempdmaxl)
                iepmintemp.extend(tempmintemp)
                iepmaxtemp.extend(tempmaxtemp)
                
                tempdatulaiks.clear()
                tempdminl.clear()
                tempdmaxl.clear()
                tempmintemp.clear()
                tempmaxtemp.clear()
                
            tabulasizmers=len(df)
            df.loc[tabulasizmers]=dati
            
            x=x+1
            
        i=i+1
        
    cikls=cikls+1
    cikli.append(cikls)
    
#tabulas pievienosana exesel faila

def datupievienosana():
    global df
    global nosauk
    global cikls
    tabula()
    fails=load_workbook(nosauk)
    Workbook = load_workbook(nosauk)
    writer = pd.ExcelWriter(nosauk, engine = 'openpyxl')
    writer.book = Workbook
    df.to_excel(writer, sheet_name = '{}. cikls'.format(cikls), index=False)
    writer.save()
    writer.close()

def laikaparveidosana():
    global laiki
    global cikls
    fails=load_workbook(nosauk)
    Workbook=load_workbook(nosauk)
    writer=pd.ExcelWriter(nosauk, engine = 'openpyxl')
    writer.book=Workbook
    worksheet=Workbook['{}. cikls'.format(cikls)]
    
    i=0
    for j in laiki:
        dttm = datetime.datetime.strptime(laiki[i], "%H:%M:%S").time()
        worksheet.cell(row=i+2,column=10).value = dttm
        i=i+1
        
    writer.save()
    writer.close()
    laiki.clear()
    
tabula()
df.to_excel(nosauk, sheet_name='{}. cikls'.format(cikls), index = False)
laikaparveidosana()
tgtlaiks=time.localtime()
formattgtlaiks=time.strftime("%H:%M",tgtlaiks)
print('{}. '.format(cikls)+' no {} cikliem'.format(ciklusk)+' ({})'.format(formattgtlaiks))

#atkartosana
j=1
while j!=ciklusk:
    print('gaida: {} minutes'.format(int(parveidlaiks/60)))
    time.sleep(parveidlaiks)
    tgtlaiks=time.localtime()
    formattgtlaiks=time.strftime("%H:%M",tgtlaiks)
    print('{}. '.format(j+1)+' no {} cikliem'.format(ciklusk)+' ({})'.format(formattgtlaiks))
    j=j+1
    datupievienosana()
    laikaparveidosana()
    
#diagrammai nepieciesamas tabulas un lapas izveide
#tabula
diagrtab={'Diagrammastabula':[]}
dfg=pd.DataFrame(diagrtab)

diagrdati='Vieta'
diagrtabulasizmers=len(dfg)
dfg.loc[diagrtabulasizmers]=diagrdati

i=0
while i!=len(cikli):
    diagrdati='Temperatūra(°){}'.format(i+1)
    diagrtabulasizmers=len(dfg)
    dfg.loc[diagrtabulasizmers]=diagrdati
    
    i=i+1

fails=load_workbook(nosauk)
Workbook=load_workbook(nosauk)
writer=pd.ExcelWriter(nosauk, engine = 'openpyxl')
writer.book=Workbook
dfg.to_excel(writer, sheet_name = 'Diagramma',header=None , index=False)

#lapas
worksheetdiag=Workbook['Diagramma']

i=0
while i!=len(cikli):
    worksheet=Workbook['{}. cikls'.format(cikli[i])]
    maxr=worksheet.max_row
    if i==0:
        for j in range(2,maxr+2):
            nolasa=worksheet.cell(row=j,column=1)
            ieliek=worksheetdiag.cell(row=1,column=j).value=nolasa.value

    for j in range(2,maxr+2):
        nolasa=worksheet.cell(row=j,column=3)
        ieliek=worksheetdiag.cell(row=2+i,column=j).value=nolasa.value

    i=i+1

#stils
thin_border = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thin'), 
                     bottom=Side(style='thin'))

i=0
while i!=len(cikli)+1:
    cell=worksheetdiag['A{}'.format(i+1)]
    cell.alignment=Alignment(horizontal='right')
    cell.font=Font(bold=True)
    cell.border=thin_border
    i=i+1

writer.save()
writer.close()

#diagrammas izveide
#datu atlase
fails=load_workbook(nosauk)
Workbook=load_workbook(nosauk)
writer=pd.ExcelWriter(nosauk, engine = 'openpyxl')
writer.book=Workbook
worksheetdiag=Workbook['Diagramma']
values=Reference(worksheetdiag, min_col=2, min_row=1, max_col=mazskaits+1, max_row=1+len(cikli))
labels=Reference(worksheetdiag, min_col=1, min_row=2, max_col=1, max_row=1+len(cikli))

#tips
chart=LineChart()
chart.add_data(values,titles_from_data=True)
chart.set_categories(labels)
from copy import deepcopy
stacked=deepcopy(chart)
stacked.grouping="stacked"
worksheetdiag.add_chart(stacked, "A{}".format(2+len(cikli)))

#stils
stacked.title="Diennakts temperaturu izmainas Latvijas pilsetas"
stacked.legend.position='b'
stacked.height=20
stacked.width=30
stacked.dataLabels=DataLabelList() 
stacked.dataLabels.showVal=True

writer.save()
writer.close()

print('')
print('Fails uztaisīts')

1.  no 1 cikliem (20:58)

Fails uztaisīts
